In [2]:
import kmapper as km 
from kmapper.jupyter import display
import umap.umap_ as umap
import sklearn
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from geopy.geocoders import Nominatim
import ssl
import certifi

In [3]:
dfinicial = pd.read_csv('COVID19MEXICO.csv')
dfinicial.dtypes

FECHA_ACTUALIZACION      object
ID_REGISTRO              object
ORIGEN                    int64
SECTOR                    int64
ENTIDAD_UM                int64
SEXO                      int64
ENTIDAD_NAC               int64
ENTIDAD_RES               int64
MUNICIPIO_RES             int64
TIPO_PACIENTE             int64
FECHA_INGRESO            object
FECHA_SINTOMAS           object
FECHA_DEF                object
INTUBADO                  int64
NEUMONIA                  int64
EDAD                      int64
NACIONALIDAD              int64
EMBARAZO                  int64
HABLA_LENGUA_INDIG        int64
INDIGENA                  int64
DIABETES                  int64
EPOC                      int64
ASMA                      int64
INMUSUPR                  int64
HIPERTENSION              int64
OTRA_COM                  int64
CARDIOVASCULAR            int64
OBESIDAD                  int64
RENAL_CRONICA             int64
TABAQUISMO                int64
OTRO_CASO                 int64
TOMA_MUE

Falta agregar: Estado (Nacimiento) Latitud y Longitud, Número de Casos Confirmados (hasta la fecha, por estado), Estado de Residencia Latitud y Longitud, Municipio de Residencia Latitud y Longitud

In [4]:
len(dfinicial)

1294293

In [5]:
df = dfinicial[["FECHA_ACTUALIZACION","ENTIDAD_NAC", "ENTIDAD_RES","MUNICIPIO_RES"]]

Borramos todos los valores nulos, estos aparecen en la base de datos como 99 o 999 según el diccionario, por lo que tenemos que tratarlos como un valor diferente de nulo.

In [6]:
df['bools'] = (df["ENTIDAD_NAC"] > 95) | (df["MUNICIPIO_RES"] > 996) | (df["ENTIDAD_RES"] > 95)
df = df.drop(df[df["bools"]==True].index)

df1 = df.drop('bools', axis=1)
df1['ID_RES'] = df1["ENTIDAD_RES"].astype(str) + df1["MUNICIPIO_RES"].astype(str)

df2 = df1[["FECHA_ACTUALIZACION", "ENTIDAD_NAC", "ID_RES"]]
df2

/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_3757/2680069795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bools'] = (df["ENTIDAD_NAC"] > 95) | (df["MUNICIPIO_RES"] > 996) | (df["ENTIDAD_RES"] > 95)


,FECHA_ACTUALIZACION,ENTIDAD_NAC,ID_RES
0,2024-04-16,15,33
1,2024-04-16,25,1817
2,2024-04-16,14,14120
3,2024-04-16,3,33
4,2024-04-16,7,7107
...,...,...,...
1294273,2024-04-16,15,15110
1294278,2024-04-16,15,1578
1294283,2024-04-16,9,916
1294288,2024-04-16,30,1533


Aqui tuve problema con la dirección de la base de datos

In [7]:
Dic_MN = pd.read_excel ('201128 Catalogos.xlsx', sheet_name ='Catálogo MUNICIPIOS') 

In [8]:
Dic_MN['bools'] = (Dic_MN["CLAVE_MUNICIPIO"] > 995) | (Dic_MN["CLAVE_ENTIDAD"] > 995)
Dic_MN = Dic_MN.drop(Dic_MN[Dic_MN["bools"]==True].index)

Dic_MN['ID'] = Dic_MN["CLAVE_ENTIDAD"].astype(str) + Dic_MN["CLAVE_MUNICIPIO"].astype(str)

Dic_MN1 = Dic_MN[["MUNICIPIO","CLAVE_ENTIDAD","ID"]]
(Dic_MN1.value_counts()==2).unique() #Para ver si no hay repetidos, efectivamente no hay repetidos

array([False])

In [9]:
Dic_MN1.head()

,MUNICIPIO,CLAVE_ENTIDAD,ID
0,AGUASCALIENTES,1,11
1,ASIENTOS,1,12
2,CALVILLO,1,13
3,COSÍO,1,14
4,JESÚS MARÍA,1,15


In [10]:
Dic_EN = pd.read_excel ('201128 Catalogos.xlsx', sheet_name ='Catálogo de ENTIDADES') 
Dic_EN['bools'] = Dic_EN["CLAVE_ENTIDAD"] > 995
Dic_EN1 = Dic_EN.drop(Dic_EN[Dic_EN["bools"]==True].index)

Dic_EN1= Dic_EN1[['ENTIDAD_FEDERATIVA','CLAVE_ENTIDAD']]

print((Dic_EN1.value_counts()==2).unique())

Dic_EN.head()

[False]


,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,bools
0,1,AGUASCALIENTES,AS,False
1,2,BAJA CALIFORNIA,BC,False
2,3,BAJA CALIFORNIA SUR,BS,False
3,4,CAMPECHE,CC,False
4,5,COAHUILA DE ZARAGOZA,CL,False


In [11]:
def obtener_estado(clave_entidad):
    estado = Dic_EN1[Dic_EN1['CLAVE_ENTIDAD'] == clave_entidad]
    if not estado.empty:
        return estado.iloc[0]  # Devuelve el primer resultado encontrado
    else:
        return None  # Devuelve None si no se encuentra ningún estado con esa clave

# Función para obtener el municipio por ID
def obtener_municipio(id_municipio):
    municipio = Dic_MN1[Dic_MN1['ID'] == str(id_municipio)]
    if not municipio.empty:
        return municipio.iloc[0]  # Devuelve el primer resultado encontrado
    else:
        return None  # Devuelve None si no se encuentra ningún municipio con ese ID


In [12]:
Dic_MN1.reset_index(drop=True, inplace=True)
Dic_MN1.head(15)

,MUNICIPIO,CLAVE_ENTIDAD,ID
0,AGUASCALIENTES,1,11
1,ASIENTOS,1,12
2,CALVILLO,1,13
3,COSÍO,1,14
4,JESÚS MARÍA,1,15
5,PABELLÓN DE ARTEAGA,1,16
6,RINCÓN DE ROMOS,1,17
7,SAN JOSÉ DE GRACIA,1,18
8,TEPEZALÁ,1,19
9,EL LLANO,1,110


In [13]:
obtener_estado(int(Dic_MN1['CLAVE_ENTIDAD'][2]))

ENTIDAD_FEDERATIVA    AGUASCALIENTES
CLAVE_ENTIDAD                      1
Name: 0, dtype: object

In [14]:
Municipio_ID = []
Entidad_ID = []

for i in range(len(Dic_MN1)):
    MunID = obtener_municipio(Dic_MN1['ID'][i])
    Municipio_ID.append(MunID)
    EntID = obtener_estado(int(Dic_MN1['CLAVE_ENTIDAD'][i]))
    Entidad_ID.append(EntID)

MunicipiosDF = pd.concat(Municipio_ID)
EntidadDF = pd.concat(Entidad_ID)

EMDF = MunicipiosDF[['MUNICIPIO']],EntidadDF[['ENTIDAD_FEDERATIVA']]

In [29]:
import time
import ssl
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

# Configure SSL context
ctx = ssl.create_default_context()

# Initialize geolocalizador
geolocalizador = Nominatim(user_agent="GEO", ssl_context=ctx)

def obtener_lat_long(nombre_municipio, nombre_estado):
    try:
        if nombre_municipio:
            ubicacion = geolocalizador.geocode(f"{nombre_municipio},{nombre_estado}, Mexico", timeout=10)
        else:
            ubicacion = geolocalizador.geocode(f"{nombre_estado}, Mexico", timeout=10)
            
        if ubicacion:
            return ubicacion.latitude, ubicacion.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        # If a timeout error occurs, wait for a moment and retry
        print("Geocoding service timed out. Retrying in 5 seconds...")
        time.sleep(5)  # Wait for 5 seconds before retrying
        return obtener_lat_long(nombre_municipio, nombre_estado)
    except GeocoderUnavailable:
        # If the geocoding service is unavailable, handle it appropriately
        print("Geocoding service is unavailable. Please try again later.")
        return None, None
    except Exception as e:
        # Handle other exceptions
        print(f"An error occurred: {str(e)}")
        return None, None



In [39]:
lat_long_df = pd.DataFrame({'Municipio': [t[0] for t in EMDF], 'Entidad': [t[1] for t in EMDF]})
lat_long_df.loc[:, ['Latitud', 'Longitud']] = lat_long_df.apply(lambda row: obtener_lat_long(row['Municipio'], row['Entidad']), axis=1)
Dic_MN1.loc[:, ['Latitud', 'Longitud']] = lat_long_df[['Latitud', 'Longitud']]


/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_3757/723237748.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat_long_df = pd.DataFrame({'Municipio': [t[0] for t in EMDF], 'Entidad': [t[1] for t in EMDF]})


Geocoding service is unavailable. Please try again later.
Geocoding service is unavailable. Please try again later.


Hata aqui hemos llegado hasta el paso 2 que es Crear la Base de Datos que contenga the date of update, state names, latitude, longitude, number of confirmed cases, state of residence, and municipality of residence aunado a la latitud y longitud del mismo

In [17]:
Dic_MN1

,MUNICIPIO,CLAVE_ENTIDAD,ID,Latitud,Longitud
0,AGUASCALIENTES,1,11,None,None
1,ASIENTOS,1,12,None,None
2,CALVILLO,1,13,NaN,NaN
3,COSÍO,1,14,NaN,NaN
4,JESÚS MARÍA,1,15,NaN,NaN
...,...,...,...,...,...
2460,VILLA HIDALGO,32,3254,NaN,NaN
2461,VILLANUEVA,32,3255,NaN,NaN
2462,ZACATECAS,32,3256,NaN,NaN
2463,TRANCOSO,32,3257,NaN,NaN


In [18]:
lat_long_df = pd.DataFrame({'Municipio': [t[0] for t in EMDF], 'Entidad': [t[1] for t in EMDF]})
lat_long_df[['Latitud EN', 'Longitud EN']] = lat_long_df.apply(lambda row: obtener_lat_long(row['Municipio'], row['Entidad']), axis=1)
Dic_MN1[['Latitud EN', 'Longitud EN']] = lat_long_df[['Latitud EN', 'Longitud EN']]

/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_3757/2423037892.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat_long_df = pd.DataFrame({'Municipio': [t[0] for t in EMDF], 'Entidad': [t[1] for t in EMDF]})


Geocoding service is unavailable. Please try again later.
Geocoding service is unavailable. Please try again later.


/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_3757/2423037892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dic_MN1[['Latitud EN', 'Longitud EN']] = lat_long_df[['Latitud EN', 'Longitud EN']]
/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_3757/2423037892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dic_MN1[['Latitud EN', 'Longitud EN']] = lat_long_df[['Latitud EN', 'Longitud EN']]


In [19]:
Dic_MN1

,MUNICIPIO,CLAVE_ENTIDAD,ID,Latitud,Longitud,Latitud EN,Longitud EN
0,AGUASCALIENTES,1,11,None,None,None,None
1,ASIENTOS,1,12,None,None,None,None
2,CALVILLO,1,13,NaN,NaN,NaN,NaN
3,COSÍO,1,14,NaN,NaN,NaN,NaN
4,JESÚS MARÍA,1,15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2460,VILLA HIDALGO,32,3254,NaN,NaN,NaN,NaN
2461,VILLANUEVA,32,3255,NaN,NaN,NaN,NaN
2462,ZACATECAS,32,3256,NaN,NaN,NaN,NaN
2463,TRANCOSO,32,3257,NaN,NaN,NaN,NaN


In [20]:
MunicipiosLL = Dic_MN1[["ID","MUNICIPIO","Latitud","Longitud","Latitud EN","Longitud EN"]]
Ent = pd.Series(list(EntidadDF['ENTIDAD_FEDERATIVA']))
MunicipiosLL1=pd.concat([MunicipiosLL,Ent], axis=1)
MunicipiosLL1 = MunicipiosLL1.rename(columns={0: 'ENTIDAD'})
MunicipiosLL1.head(15)

,ID,MUNICIPIO,Latitud,Longitud,Latitud EN,Longitud EN,ENTIDAD
0,11,AGUASCALIENTES,None,None,None,None,AGUASCALIENTES
1,12,ASIENTOS,None,None,None,None,AGUASCALIENTES
2,13,CALVILLO,NaN,NaN,NaN,NaN,AGUASCALIENTES
3,14,COSÍO,NaN,NaN,NaN,NaN,AGUASCALIENTES
4,15,JESÚS MARÍA,NaN,NaN,NaN,NaN,AGUASCALIENTES
5,16,PABELLÓN DE ARTEAGA,NaN,NaN,NaN,NaN,AGUASCALIENTES
6,17,RINCÓN DE ROMOS,NaN,NaN,NaN,NaN,AGUASCALIENTES
7,18,SAN JOSÉ DE GRACIA,NaN,NaN,NaN,NaN,AGUASCALIENTES
8,19,TEPEZALÁ,NaN,NaN,NaN,NaN,AGUASCALIENTES
9,110,EL LLANO,NaN,NaN,NaN,NaN,AGUASCALIENTES


In [21]:
df3 = df2.merge(MunicipiosLL1, how='inner', left_on='ID_RES', right_on='ID')

In [22]:
df3.reset_index(drop=True, inplace=True)
df3

,FECHA_ACTUALIZACION,ENTIDAD_NAC,ID_RES,ID,MUNICIPIO,Latitud,Longitud,Latitud EN,Longitud EN,ENTIDAD
0,2024-04-16,15,33,33,LA PAZ,NaN,NaN,NaN,NaN,BAJA CALIFORNIA SUR
1,2024-04-16,25,1817,1817,TEPIC,NaN,NaN,NaN,NaN,NAYARIT
2,2024-04-16,14,14120,14120,ZAPOPAN,NaN,NaN,NaN,NaN,JALISCO
3,2024-04-16,3,33,33,LA PAZ,NaN,NaN,NaN,NaN,BAJA CALIFORNIA SUR
4,2024-04-16,7,7107,7107,VILLA CORZO,NaN,NaN,NaN,NaN,CHIAPAS
...,...,...,...,...,...,...,...,...,...,...
1285365,2024-04-16,15,15110,15110,VALLE DE BRAVO,NaN,NaN,NaN,NaN,MÉXICO
1285366,2024-04-16,15,1578,1578,SANTO TOMÁS,NaN,NaN,NaN,NaN,MÉXICO
1285367,2024-04-16,9,916,916,MIGUEL HIDALGO,NaN,NaN,NaN,NaN,CIUDAD DE MÉXICO
1285368,2024-04-16,30,1533,1533,ECATEPEC DE MORELOS,NaN,NaN,NaN,NaN,MÉXICO


In [23]:
df4 = df3[['ID','ENTIDAD','Latitud EN', 'Longitud EN', 'MUNICIPIO', 'Latitud', 'Longitud']]
df4

,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud
0,33,BAJA CALIFORNIA SUR,NaN,NaN,LA PAZ,NaN,NaN
1,1817,NAYARIT,NaN,NaN,TEPIC,NaN,NaN
2,14120,JALISCO,NaN,NaN,ZAPOPAN,NaN,NaN
3,33,BAJA CALIFORNIA SUR,NaN,NaN,LA PAZ,NaN,NaN
4,7107,CHIAPAS,NaN,NaN,VILLA CORZO,NaN,NaN
...,...,...,...,...,...,...,...
1285365,15110,MÉXICO,NaN,NaN,VALLE DE BRAVO,NaN,NaN
1285366,1578,MÉXICO,NaN,NaN,SANTO TOMÁS,NaN,NaN
1285367,916,CIUDAD DE MÉXICO,NaN,NaN,MIGUEL HIDALGO,NaN,NaN
1285368,1533,MÉXICO,NaN,NaN,ECATEPEC DE MORELOS,NaN,NaN


In [24]:
latitudes = []
longitudes = []

for i in range(len(Dic_EN1)):
    latitud, longitud = obtener_lat_long(None, Dic_EN1['ENTIDAD_FEDERATIVA'][i])
    latitudes.append(latitud)
    longitudes.append(longitud)

Dic_EN1["Latitud EN NA"] = latitudes
Dic_EN1["Longitud EN NA"] = longitudes
Dic_EN1.head()

Geocoding service is unavailable. Please try again later.
Geocoding service is unavailable. Please try again later.


KeyboardInterrupt: 

In [ ]:
#Re = []

#for i in range(len(df2)):
#    Res = Dic_EN1[Dic_EN1['CLAVE_ENTIDAD'] == df2['ENTIDAD_NAC'][i]]
#    Re.append(Res)

Entss = df2.merge(Dic_EN1, how='left', left_on='ENTIDAD_NAC', right_on='CLAVE_ENTIDAD')
Entss

,FECHA_ACTUALIZACION,ENTIDAD_NAC,ID_RES,ENTIDAD_FEDERATIVA,CLAVE_ENTIDAD,Latitud EN NA,Longitud EN NA
0,2024-04-16,15,33,MÉXICO,15,19.432630,-99.133178
1,2024-04-16,25,1817,SINALOA,25,25.000000,-107.500000
2,2024-04-16,14,14120,JALISCO,14,20.333333,-103.666667
3,2024-04-16,3,33,BAJA CALIFORNIA SUR,3,25.581801,-111.570616
4,2024-04-16,7,7107,CHIAPAS,7,16.500000,-92.500000
...,...,...,...,...,...,...,...
1284726,2024-04-16,15,15110,MÉXICO,15,19.432630,-99.133178
1284727,2024-04-16,15,1578,MÉXICO,15,19.432630,-99.133178
1284728,2024-04-16,9,916,CIUDAD DE MÉXICO,9,19.432630,-99.133178
1284729,2024-04-16,30,1533,VERACRUZ DE IGNACIO DE LA LLAVE,30,19.333333,-96.666667


In [ ]:
Entss

,ENTIDAD_FEDERATIVA,CLAVE_ENTIDAD,Latitud EN NA,Longitud EN NA
0,MÉXICO,15,19.432630,-99.133178
1,SINALOA,25,25.000000,-107.500000
2,JALISCO,14,20.333333,-103.666667
3,BAJA CALIFORNIA SUR,3,25.581801,-111.570616
4,CHIAPAS,7,16.500000,-92.500000
...,...,...,...,...
1284726,MÉXICO,15,19.432630,-99.133178
1284727,MÉXICO,15,19.432630,-99.133178
1284728,CIUDAD DE MÉXICO,9,19.432630,-99.133178
1284729,VERACRUZ DE IGNACIO DE LA LLAVE,30,19.333333,-96.666667


In [ ]:
Entss.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud
0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853
1,1817,NAYARIT,22.000000,-105.000000,TEPIC,21.511916,-104.891589
2,14120,JALISCO,20.333333,-103.666667,ZAPOPAN,20.721120,-103.391367
3,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853
4,7107,CHIAPAS,16.500000,-92.500000,VILLA CORZO,16.063189,-93.265003
...,...,...,...,...,...,...,...
1285365,15110,MÉXICO,19.432630,-99.133178,VALLE DE BRAVO,19.194264,-100.131663
1285366,1578,MÉXICO,19.432630,-99.133178,SANTO TOMÁS,19.607524,-99.048691
1285367,916,CIUDAD DE MÉXICO,19.432630,-99.133178,MIGUEL HIDALGO,19.407298,-99.190671
1285368,1533,MÉXICO,19.432630,-99.133178,ECATEPEC DE MORELOS,19.599408,-99.048728


In [ ]:
DFF=pd.concat([df4,Entss], axis=1)
DFF = DFF.drop ('CLAVE_ENTIDAD', axis = 1)
DFF = DFF.rename(columns={'ENTIDAD_FEDERATIVA': 'ENTIDAD NA'})
DFF.reset_index(drop=True, inplace=True)
DFF.dropna()
DFF

,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud,FECHA_ACTUALIZACION,ENTIDAD_NAC,ID_RES,ENTIDAD NA,Latitud EN NA,Longitud EN NA
0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,2024-04-16,15.0,33,MÉXICO,19.432630,-99.133178
1,1817,NAYARIT,22.000000,-105.000000,TEPIC,21.511916,-104.891589,2024-04-16,25.0,1817,SINALOA,25.000000,-107.500000
2,14120,JALISCO,20.333333,-103.666667,ZAPOPAN,20.721120,-103.391367,2024-04-16,14.0,14120,JALISCO,20.333333,-103.666667
3,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,2024-04-16,3.0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616
4,7107,CHIAPAS,16.500000,-92.500000,VILLA CORZO,16.063189,-93.265003,2024-04-16,7.0,7107,CHIAPAS,16.500000,-92.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285365,15110,MÉXICO,19.432630,-99.133178,VALLE DE BRAVO,19.194264,-100.131663,NaN,NaN,NaN,NaN,NaN,NaN
1285366,1578,MÉXICO,19.432630,-99.133178,SANTO TOMÁS,19.607524,-99.048691,NaN,NaN,NaN,NaN,NaN,NaN
1285367,916,CIUDAD DE MÉXICO,19.432630,-99.133178,MIGUEL HIDALGO,19.407298,-99.190671,NaN,NaN,NaN,NaN,NaN,NaN
1285368,1533,MÉXICO,19.432630,-99.133178,ECATEPEC DE MORELOS,19.599408,-99.048728,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
DFF1=pd.concat([DFF,df2], axis=1)
DFF1 = DFF1.drop (['ID_RES','ENTIDAD_NAC'], axis = 1)
DFF1

,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud,ENTIDAD NA,Latitud EN NA,Longitud EN NA,FECHA_ACTUALIZACION
0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,MÉXICO,19.432630,-99.133178,2024-04-16
1,1817,NAYARIT,22.000000,-105.000000,TEPIC,21.511916,-104.891589,SINALOA,25.000000,-107.500000,2024-04-16
2,14120,JALISCO,20.333333,-103.666667,ZAPOPAN,20.721120,-103.391367,JALISCO,20.333333,-103.666667,2024-04-16
3,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,BAJA CALIFORNIA SUR,25.581801,-111.570616,2024-04-16
4,7107,CHIAPAS,16.500000,-92.500000,VILLA CORZO,16.063189,-93.265003,CHIAPAS,16.500000,-92.500000,2024-04-16
...,...,...,...,...,...,...,...,...,...,...,...
1285365,15110,MÉXICO,19.432630,-99.133178,VALLE DE BRAVO,19.194264,-100.131663,NaN,NaN,NaN,NaN
1285366,1578,MÉXICO,19.432630,-99.133178,SANTO TOMÁS,19.607524,-99.048691,NaN,NaN,NaN,NaN
1285367,916,CIUDAD DE MÉXICO,19.432630,-99.133178,MIGUEL HIDALGO,19.407298,-99.190671,NaN,NaN,NaN,NaN
1285368,1533,MÉXICO,19.432630,-99.133178,ECATEPEC DE MORELOS,19.599408,-99.048728,NaN,NaN,NaN,NaN


In [ ]:
# Primero, asegúrate de tener el nombre correcto del DataFrame, asumiendo que es DFF1
# Si estás utilizando un DataFrame llamado df en lugar de DFF1, asegúrate de cambiarlo a DFF1

# Resetear el índice del DataFrame DFF1
DFF1.reset_index(drop=True, inplace=True)

# Definir el rango de índices a eliminar
inicio = 1284730
fin = 1285368  # No incluyas el último índice del rango

# Eliminar las filas dentro del rango especificado
DFF1.drop(range(inicio, fin + 2), inplace=True)

# Mostrar el DataFrame resultante
DFF1


,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud,ENTIDAD NA,Latitud EN NA,Longitud EN NA,FECHA_ACTUALIZACION
0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,MÉXICO,19.432630,-99.133178,2024-04-16
1,1817,NAYARIT,22.000000,-105.000000,TEPIC,21.511916,-104.891589,SINALOA,25.000000,-107.500000,2024-04-16
2,14120,JALISCO,20.333333,-103.666667,ZAPOPAN,20.721120,-103.391367,JALISCO,20.333333,-103.666667,2024-04-16
3,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,BAJA CALIFORNIA SUR,25.581801,-111.570616,2024-04-16
4,7107,CHIAPAS,16.500000,-92.500000,VILLA CORZO,16.063189,-93.265003,CHIAPAS,16.500000,-92.500000,2024-04-16
...,...,...,...,...,...,...,...,...,...,...,...
1284725,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,PUEBLA,18.833333,-98.000000,2024-04-16
1284726,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,MÉXICO,19.432630,-99.133178,2024-04-16
1284727,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,MÉXICO,19.432630,-99.133178,2024-04-16
1284728,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,CIUDAD DE MÉXICO,19.432630,-99.133178,2024-04-16


In [ ]:
DFF1['FECHA_ACTUALIZACION'] = pd.to_datetime(DFF1['FECHA_ACTUALIZACION'], format='%Y-%m-%d', errors='coerce')
DFF1

,ID,ENTIDAD,Latitud EN,Longitud EN,MUNICIPIO,Latitud,Longitud,ENTIDAD NA,Latitud EN NA,Longitud EN NA,FECHA_ACTUALIZACION
0,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,MÉXICO,19.432630,-99.133178,2024-04-16
1,1817,NAYARIT,22.000000,-105.000000,TEPIC,21.511916,-104.891589,SINALOA,25.000000,-107.500000,2024-04-16
2,14120,JALISCO,20.333333,-103.666667,ZAPOPAN,20.721120,-103.391367,JALISCO,20.333333,-103.666667,2024-04-16
3,33,BAJA CALIFORNIA SUR,25.581801,-111.570616,LA PAZ,24.161995,-110.315853,BAJA CALIFORNIA SUR,25.581801,-111.570616,2024-04-16
4,7107,CHIAPAS,16.500000,-92.500000,VILLA CORZO,16.063189,-93.265003,CHIAPAS,16.500000,-92.500000,2024-04-16
...,...,...,...,...,...,...,...,...,...,...,...
1284725,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,PUEBLA,18.833333,-98.000000,2024-04-16
1284726,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,MÉXICO,19.432630,-99.133178,2024-04-16
1284727,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,MÉXICO,19.432630,-99.133178,2024-04-16
1284728,1546,MÉXICO,19.432630,-99.133178,JILOTZINGO,19.497769,-99.395578,CIUDAD DE MÉXICO,19.432630,-99.133178,2024-04-16


In [ ]:
mapper = km.KeplerMapper(verbose=1)


KeplerMapper(verbose=1)


In [ ]:
# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(DFF1)#aqui debe de ir nuestra data nueva


ValueError: could not convert string to float: 'BAJA CALIFORNIA SUR'

In [ ]:
# PCA for projection
from sklearn.decomposition import PCA
projection = PCA(n_components=2)
projected_data = projection.fit_transform(data_scaled)


NameError: name 'data_scaled' is not defined

In [ ]:
# Define the cover and the clustering algorithm
cover = km.Cover(n_cubes=10, perc_overlap=0.8)
clusterer = DBSCAN()

# Create the simplicial complex
graph = mapper.map(projected_data, inverse_X=data_scaled, cover=cover, clusterer=clusterer)


NameError: name 'DBSCAN' is not defined

In [ ]:
# Visualize it
mapper.visualize(graph, path_html="covid19-mapper-output.html",
                 title="COVID-19 Data Mapper")


NameError: name 'graph' is not defined

4. Realicen un conver de 10 cubos y un overlap de 8\% con la métrica Euclidiana.

In [ ]:
import kmapper as km
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

# Initialize KeplerMapper
mapper = km.KeplerMapper(verbose=1)

# Fit and transform your data with scaling if necessary
# Assuming 'Dic_MN1' contains the data you want to analyze
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(Dic_MN1[['Latitud', 'Longitud']])  # Scale only Latitud and Longitud

# Define the cover with 10 cubes and 8% overlap
cover = km.Cover(n_cubes=10, perc_overlap=0.08)

# Choose the clustering algorithm, in this case, DBSCAN
clusterer = DBSCAN(eps=0.5, min_samples=5)  # eps and min_samples are hyperparameters you may need to tune

# Define a projection. This can be a dimensionality reduction algorithm or a feature extraction method.
# For simple cases, you can use the scaled data directly
projection = data_scaled

# Create the simplicial complex
complex = mapper.map(projection,
                     X=data_scaled,  # the scaled data
                     cover=cover,
                     clusterer=clusterer)

# Visualize the complex
mapper.visualize(complex, path_html="mapper_visualization_output.html", title="KeplerMapper on COVID-19 Data")


KeplerMapper(verbose=1)
Mapping on data shaped (2465, 2) using lens shaped (2465, 2)

Creating 100 hypercubes.

Created 0 edges and 0 nodes in 0:00:00.012006.


Exception: Visualize requires a mapper with more than 0 nodes. 
It is possible that the constructed mapper could have been constructed with bad parameters. This can occasionally happens when using the default clustering algorithm. Try changing `eps` or `min_samples` in the DBSCAN clustering algorithm.

n_cubes refers to the number of intervals (or hypercubes) you want to create along each dimension in the projection.
perc_overlap refers to how much each cube should overlap with the next. An overlap of 0.08 means that there is an 8% overlap.
The DBSCAN parameters eps and min_samples are hyperparameters for the clustering algorithm that you may need to tune based on your data.
projection in this context refers to any transformation of your data to a space where the topology becomes more apparent. This could be a simple scaling, or it could involve a more complex transformation like PCA, t-SNE, or UMAP. For the actual mapper.map method, you can directly pass the projection if you are using a dimensionality reduction technique or just the data itself if it's already suitable for analysis.
Finally, mapper.visualize creates an interactive HTML visualization of the simplicial complex.

5. Utilicen DBSCAN para identificar clusters.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

# Assuming Dic_MN1 contains the 'Latitud' and 'Longitud' you want to cluster
# First, scale your data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(Dic_MN1[['Latitud', 'Longitud']])

# Initialize DBSCAN with appropriate parameters
# The eps and min_samples need to be chosen based on your specific dataset
dbscan = DBSCAN(eps=0.5, min_samples=5)  # eps: maximum distance between two samples, min_samples: number of samples in a neighborhood for a point to be considered a core point

# Fit the DBSCAN algorithm to your data
clusters = dbscan.fit_predict(data_scaled)

# Add the cluster labels to your original DataFrame
Dic_MN1['Cluster'] = clusters

# Now, Dic_MN1 has a new column 'Cluster' with cluster labels assigned by DBSCAN


ValueError: Input X contains NaN.
DBSCAN does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Al momento de querer juntar df2 y Dic_MN1.head() para realizar el maper crasheo. 